In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, Model

In [ ]:
import shap

In [ ]:
%pip install deep 

  Preparing metadata (setup.py) ... done
  Created wheel for deep: filename=deep-0.10-py3-none-any.whl size=7602 sha256=b7bae979a2b3f7ab355318d9b0f18eb70a2b15f9ee62399007486dc9dac62b0f
  Stored in directory: /root/.cache/pip/wheels/f3/35/e4/febc4b6bdb9a0d0e9f2577c2f4eee7097eab08af70fbaa1a73
Successfully built deep


In [ ]:
import zipfile
with zipfile.ZipFile('D37.zip', 'r') as zip_ref:
    zip_ref.extractall('')

FileNotFoundError: [Errno 2] No such file or directory: 'D37.zip'

In [ ]:
df = pd.read_csv('final.csv',low_memory=False)
df = df.drop(['Unnamed: 0', 'Year', 'DepartureDelayGroups', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'DivAirportLandings', 'wpgt', 'tsun'], axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: 'final.csv'

In [ ]:
df.columns

Index(['FlightDate', 'DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'index', 'tavg', 'tmin', 'tmax', 'prcp',
       'snow', 'wdir', 'wspd', 'pres'],
      dtype='object')

In [ ]:
df = df.drop(['index'], axis = 1)

In [ ]:
grouped = df.groupby(['FlightDate', 'OriginCityName'])

# Fill NaN in Col1 with the mean of matching groups
for col in ['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'pres']:
    df[col] = grouped[col].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.isna().sum()

FlightDate                              0
DOT_ID_Reporting_Airline                0
Tail_Number                          1989
Flight_Number_Reporting_Airline         0
OriginAirportID                         0
OriginCityName                          0
OriginWac                               0
DestAirportID                           0
DestCityName                            0
DestWac                                 0
CRSDepTime                              0
DepTime                              9945
DepDelayMinutes                      9953
CRSArrTime                              0
ArrTime                             10419
ArrDelayMinutes                     11676
Diverted                                0
CRSElapsedTime                          0
ActualElapsedTime                   11676
AirTime                             11676
Flights                                 0
Distance                                0
CarrierDelay                       431997
WeatherDelay                      

In [ ]:
df = df.drop(['snow'], axis=1)
df = df.dropna()
df

,FlightDate,DOT_ID_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginCityName,OriginWac,DestAirportID,DestCityName,DestWac,...,SecurityDelay,LateAircraftDelay,CancellationCode_encoded,tavg,tmin,tmax,prcp,wdir,wspd,pres
4,2024-01-31,20452,N642RW,3493,12264,"Washington, DC",38,10785,"Burlington, VT",16,...,0.0,0.0,0,4.9,2.8,6.7,0.0,33.0,6.8,1019.2
5,2024-01-15,19790,N590NW,1204,11433,"Detroit, MI",43,14635,"Fort Myers, FL",33,...,0.0,58.0,0,-15.3,-18.5,-11.9,0.0,234.0,21.8,1023.6
7,2024-01-05,20368,333NV,292,14262,"Palm Springs, CA",91,10666,"Bellingham, WA",93,...,0.0,1.0,0,13.8,5.6,21.7,0.0,326.0,8.5,1020.4
10,2024-01-13,19790,N921AT,2459,12197,"White Plains, NY",22,14027,"West Palm Beach/Palm Beach, FL",33,...,0.0,0.0,0,7.9,0.6,13.9,35.3,226.0,21.8,995.3
17,2024-01-09,19393,N255WN,1670,14986,"Sarasota/Bradenton, FL",33,11259,"Dallas, TX",74,...,0.0,366.0,0,21.2,17.2,25.0,7.2,178.0,30.8,1010.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546904,2022-12-24,20304,N730EV,3226,13930,"Chicago, IL",41,12197,"White Plains, NY",22,...,0.0,0.0,0,22.9,20.0,27.0,0.2,186.0,16.9,1015.3
546907,2022-12-01,20397,N592NN,5087,11278,"Washington, DC",38,13198,"Kansas City, MO",64,...,0.0,21.0,0,-7.5,-20.5,4.4,4.1,237.0,29.2,1002.0
546917,2022-12-29,20416,N921NK,2410,11618,"Newark, NJ",21,13796,"Oakland, CA",91,...,0.0,14.0,0,19.7,16.6,23.1,0.0,43.0,17.1,1023.8
546921,2022-12-23,19790,N897DN,2554,12892,"Los Angeles, CA",91,13303,"Miami, FL",33,...,0.0,32.0,0,23.1,21.0,27.0,0.1,170.0,14.1,1021.4


In [ ]:
df['DateObjects'] = pd.to_datetime(df['FlightDate'])

In [ ]:
df.columns

Index(['FlightDate', 'DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', 'DateObjects'],
      dtype='object')

In [ ]:
df = df.sort_values(by='DateObjects')

In [ ]:
df = df.drop(['FlightDate'], axis = 1)
df['Year'] = df['DateObjects'].dt.year
df['Month'] = df['DateObjects'].dt.month
df['Day'] = df['DateObjects'].dt.day
df['Quarter'] = df['DateObjects'].dt.quarter
df['Week'] = df['DateObjects'].dt.isocalendar().week

In [ ]:
cut_off_date = pd.to_datetime('2024-01-01')

# Split the DataFrame into training and testing sets
train_df = df[df['DateObjects'] < cut_off_date]  # Training data
test_df = df[df['DateObjects'] >= cut_off_date]

In [ ]:
df.columns

Index(['DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'OriginWac', 'DestAirportID', 'DestCityName', 'DestWac', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime',
       'AirTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', 'DateObjects', 'Year', 'Month', 'Day', 'Quarter',
       'Week'],
      dtype='object')

In [ ]:
test_df=test_df.drop(columns=['DateObjects','OriginWac','DestWac','DepTime', 'DepDelayMinutes', 'ArrTime',
       'ArrDelayMinutes','ActualElapsedTime'])
train_df=train_df.drop(columns=['DateObjects','OriginWac','DestWac','DepTime', 'DepDelayMinutes', 'ArrTime',
       'ArrDelayMinutes','ActualElapsedTime'])

In [ ]:
train_df=train_df.drop(columns=['AirTime'])

In [ ]:
test_df=test_df.drop(columns=['AirTime'])

In [ ]:
train_df.columns

Index(['DOT_ID_Reporting_Airline', 'Tail_Number',
       'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginCityName',
       'DestAirportID', 'DestCityName', 'CRSDepTime', 'CRSArrTime', 'Diverted',
       'CRSElapsedTime', 'Flights', 'Distance', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
       'CancellationCode_encoded', 'tavg', 'tmin', 'tmax', 'prcp', 'wdir',
       'wspd', 'pres', 'Year', 'Month', 'Day', 'Quarter', 'Week'],
      dtype='object')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def prepare_data(df_new):
    # Define input and output features
    input_features = ['Year', 'DOT_ID_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline',
                      'OriginAirportID', 'OriginCityName', 'DestAirportID', 'DestCityName', 'CRSDepTime',
                      'CRSArrTime', 'CRSElapsedTime', 'Flights', 'Distance', 'tavg', 'tmin', 'tmax', 'prcp',
                      'wdir', 'wspd', 'pres', 'Month', 'Day', 'Quarter', 'Week']
    output_features = ['Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
                       'LateAircraftDelay', 'CancellationCode_encoded']

    # Separate categorical and continuous columns
    categorical_cols = ['Tail_Number', 'OriginCityName', 'DestCityName']
    continuous_cols = [col for col in input_features if col not in categorical_cols]

    # Handle missing values
    df_new = df_new.copy()
    for col in continuous_cols:
        df_new[col] = df_new[col].fillna(df_new[col].mean())
    for col in categorical_cols:
        df_new[col] = df_new[col].fillna('MISSING')
    for col in output_features:
        df_new[col] = df_new[col].fillna(0)

    # Process categorical features
    label_encoders = {}
    categorical_data = []
    vocab_sizes = []

    for col in categorical_cols:
        le = LabelEncoder()
        df_new[col] = le.fit_transform(df_new[col].astype(str))
        label_encoders[col] = le
        categorical_data.append(df_new[col].values)
        vocab_sizes.append(len(le.classes_) + 1)  # +1 for padding

    # Convert categorical data to tensor format
    categorical_tensor = torch.tensor(np.stack(categorical_data, axis=1), dtype=torch.long)

    # Normalize continuous features
    scaler = StandardScaler()
    continuous_tensor = torch.tensor(
        scaler.fit_transform(df_new[continuous_cols].values),
        dtype=torch.float32
    )

    # Convert output features to tensor
    output_tensor = torch.tensor(df_new[output_features].values, dtype=torch.float32)

    return continuous_tensor, categorical_tensor, output_tensor, continuous_cols, categorical_cols, vocab_sizes, label_encoders


In [ ]:
class CNNFlightDelayModel(nn.Module):
    def __init__(self, continuous_dim, vocab_sizes, hidden_dim, num_outputs, embedding_dim=8):
        super(CNNFlightDelayModel, self).__init__()

        # Embeddings for categorical features
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
            for vocab_size in vocab_sizes
        ])

        # Calculate total embedding dimension
        total_embedding_dim = len(vocab_sizes) * embedding_dim

        # 1D Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=continuous_dim + total_embedding_dim, out_channels=hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim // 2, kernel_size=3, padding=1)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim // 2, num_outputs)

    def forward(self, continuous_x, categorical_x):
        # Process categorical features
        embeddings = [embedding_layer(categorical_x[:, i]) for i, embedding_layer in enumerate(self.embeddings)]

        # Concatenate all embeddings
        embedded_categorical = torch.cat(embeddings, dim=1)

        # Concatenate with continuous features
        combined_input = torch.cat([continuous_x, embedded_categorical], dim=1).unsqueeze(2)  # Add channel dimension

        # Pass through convolutional layers
        conv_out = self.conv1(combined_input)
        conv_out = nn.ReLU()(conv_out)
        conv_out = self.conv2(conv_out)
        conv_out = nn.ReLU()(conv_out)

        # Flatten for fully connected layer
        conv_out = conv_out.view(conv_out.size(0), -1)

        # Output layer
        outputs = self.fc(conv_out)
        return outputs


In [ ]:
class FlightDataset(Dataset):
    def __init__(self, continuous_data, categorical_data, targets):
        self.continuous_data = continuous_data
        self.categorical_data = categorical_data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return (
            self.continuous_data[idx],
            self.categorical_data[idx],
            self.targets[idx]
        )

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=25):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for continuous_x, categorical_x, targets in train_loader:
            continuous_x = continuous_x.to(device)
            categorical_x = categorical_x.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(continuous_x, categorical_x)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for continuous_x, categorical_x, targets in test_loader:
            continuous_x = continuous_x.to(device)
            categorical_x = categorical_x.to(device)
            outputs = model(continuous_x, categorical_x)
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())

    predictions = np.concatenate(predictions)
    actuals = np.concatenate(actuals)

    # Calculate MAE and RMSE
    mae = mean_absolute_error(actuals, predictions)
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')

    return predictions, actuals, mae, rmse


In [ ]:
def main_cnn(df_new, batch_size=64, hidden_dim=128, learning_rate=0.01, num_epochs=25, embedding_dim=8):
    # Prepare data (using the same prepare_data function)
    continuous_data, categorical_data, targets, continuous_cols, categorical_cols, vocab_sizes, label_encoders = prepare_data(df_new)

    # Create dataset and dataloader
    dataset = FlightDataset(continuous_data, categorical_data, targets)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize CNN model
    model = CNNFlightDelayModel(
        continuous_dim=len(continuous_cols),
        vocab_sizes=vocab_sizes,
        hidden_dim=hidden_dim,
        num_outputs=targets.shape[1],
        embedding_dim=embedding_dim
    ).to(device)

    # Setup training
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train and evaluate
    train_model(model, train_loader, criterion, optimizer, device, num_epochs)

    # Evaluate model on the same loader (for simplicity; use separate loaders for train/test in practice)
    predictions, actuals, mae, rmse = evaluate_model(model, train_loader, device)
    print(f"Final MAE: {mae:.4f}")

    sample_data, _, sample_targets = next(iter(train_loader))
    sample_data = sample_data.to(device)

    # SHAP explanations (using KernelExplainer due to model type)
    def model_predict(data):
        data = torch.tensor(data, dtype=torch.float32).to(device)
        with torch.no_grad():
            preds = model(data).cpu().numpy()
        return preds

    # KernelExplainer setup (KernelExplainer is model-agnostic, hence can be used here)
    explainer = shap.DeepExplainer(model, sample_data.cpu().numpy())

    # Calculate SHAP values
    shap_values = explainer.shap_values(sample_data.cpu().numpy(), nsamples=100)

    # Plot explanations for the first prediction
    shap.initjs()
    shap.force_plot(explainer.expected_value[0], shap_values[0], sample_data.cpu().numpy()[0])


    return model, predictions, actuals

In [ ]:
model, predictions, actuals = main_cnn(train_df)
model, predictions, actuals = main_cnn(test_df)

Using device: cpu
Epoch [1/25], Average Loss: 1630.1449
Epoch [2/25], Average Loss: 1601.3180
Epoch [3/25], Average Loss: 1560.4637
Epoch [4/25], Average Loss: 1511.9534
Epoch [5/25], Average Loss: 1451.6555
Epoch [6/25], Average Loss: 1363.9164
Epoch [7/25], Average Loss: 1275.9186
Epoch [8/25], Average Loss: 1186.7931
Epoch [9/25], Average Loss: 1102.8642
Epoch [10/25], Average Loss: 1030.6245
Epoch [11/25], Average Loss: 959.3210
Epoch [12/25], Average Loss: 922.8868
Epoch [13/25], Average Loss: 872.3222
Epoch [14/25], Average Loss: 842.8817
Epoch [15/25], Average Loss: 817.0049
Epoch [16/25], Average Loss: 792.8978
Epoch [17/25], Average Loss: 773.6443
Epoch [18/25], Average Loss: 744.6529
Epoch [19/25], Average Loss: 733.0222
Epoch [20/25], Average Loss: 716.0733
Epoch [21/25], Average Loss: 706.9033
Epoch [22/25], Average Loss: 692.9000
Epoch [23/25], Average Loss: 685.2252
Epoch [24/25], Average Loss: 674.0138
Epoch [25/25], Average Loss: 659.6665
Mean Absolute Error (MAE): 9.04

TypeError: CNNFlightDelayModel.forward() missing 1 required positional argument: 'categorical_x'